In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, RobustScaler, FunctionTransformer
import category_encoders as ce

In [2]:
import dagshub
dagshub.init(repo_owner='sourav664', repo_name='real-estate-hybrid-app', mlflow=True)


Accessing as sourav664

Initialized MLflow to track repo "sourav664/real-estate-hybrid-app"

Repository sourav664/real-estate-hybrid-app initialized!

In [3]:
import mlflow
mlflow.set_tracking_uri("https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow")

In [4]:
# mlflow experiment

mlflow.set_experiment("Exp 4 - Stacking Regression Hyperparameter Tuning")

<Experiment: artifact_location='mlflow-artifacts:/545af2b41f364ce48ccdc6697a5c8661', creation_time=1764824733073, experiment_id='4', last_update_time=1764824733073, lifecycle_stage='active', name='Exp 4 - Stacking Regression Hyperparameter Tuning', tags={}>

In [5]:
from pathlib import Path

path = Path.cwd().parent
data_path = path / 'data/raw'

In [7]:
df_one = pd.read_csv(f"{data_path}/final_dataV4.csv")

In [8]:
predictors = ['propertytype', 'bedrooms', 'bathrooms', 'balconies', 'furnished', 'transactiontype', 'ageofcons', 'additionalRooms','region',
              'locality','superbuiltupareasqft','totalfloornumber','price']

In [9]:
df = df_one[predictors]

In [10]:
df.duplicated().sum(
    
)

np.int64(2130)

In [11]:
df.drop_duplicates(inplace=True)

C:\Users\soura\AppData\Local\Temp\ipykernel_23244\3006716147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [12]:
X = df.drop(columns=["price"])
y = df["price"]

In [13]:
# train test split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
     

In [14]:
print("The size of train data is",X_train.shape)
print("The shape of test data is",X_test.shape)

The size of train data is (31784, 12)
The shape of test data is (7946, 12)


In [15]:
ohe_encode = ['transactiontype','region','propertytype','furnished','ageofcons']
target_encode = ['locality']
robust_scaler = ['bedrooms','bathrooms','balconies','superbuiltupareasqft']

In [16]:
from sklearn.preprocessing import FunctionTransformer

pt = FunctionTransformer(np.log1p, inverse_func=np.expm1, validate=True)


In [17]:
preprocessor = ColumnTransformer(
    [
        ('ohe_encode', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), ohe_encode),
        ('robust_scaler', RobustScaler(), robust_scaler)
    ],remainder="passthrough", n_jobs=-1, force_int_remainder_cols=False,verbose_feature_names_out=False
)


preprocessor.set_output(transform="pandas")

,transformers,"[('ohe_encode', ...), ('robust_scaler', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,-1
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False
,force_int_remainder_cols,False
,categories,'auto'
,drop,'first'
,sparse_output,False


In [18]:
from lightgbm import LGBMRegressor
import optuna
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.compose import TransformedTargetRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

c:\Users\soura\OneDrive\Desktop\real-estate-hybrid-app\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
# build the best models

best_rf_params = {'n_estimators': 170,
 'max_depth': 30,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': None,
 'max_samples': 0.9932722804661284}



best_lgbm_params = {'n_estimators': 498,
 'max_depth': 9,
 'learning_rate': 0.07041175194094815,
 'subsample': 0.9404689293367969,
 'min_child_weight': 6,
 'min_split_gain': 0.0004206922397676019,
 'reg_lambda': 14.050384510957326}


best_rf = RandomForestRegressor(**best_rf_params)
best_lgbm = LGBMRegressor(**best_lgbm_params)

In [20]:
def objective(trial):
    with mlflow.start_run(nested=True):
        
       
        meta_model_name = trial.suggest_categorical("model",["LR","KNN","DT"])

        if meta_model_name == "LR":
            intercept_lr = trial.suggest_categorical("intercept_lr",[True,False])
            meta = LinearRegression(fit_intercept=intercept_lr)

        elif meta_model_name == "KNN":
            n_neighbors_knn = trial.suggest_int("n_neighbors_knn",1,15)
            weights_knn = trial.suggest_categorical("weights_knn",["uniform","distance"])
            meta = KNeighborsRegressor(n_neighbors=n_neighbors_knn,weights=weights_knn,n_jobs=-1)
        elif meta_model_name == "DT":
            max_depth_dt = trial.suggest_int("max_depth_dt",1,10)
            min_samples_split_dt = trial.suggest_int("min_samples_split_dt",2,10)
            min_samples_leaf_dt = trial.suggest_int("min_samples_leaf_dt",1,10)
            meta = DecisionTreeRegressor(max_depth=max_depth_dt,
                                        min_samples_split=min_samples_split_dt,
                                        min_samples_leaf=min_samples_leaf_dt,
                                        random_state=42)

        
        
        
     
     
        # log meta params
        mlflow.log_params(meta.get_params())
        
        
        
        
        

        # stacking regressor
        stacking_reg = StackingRegressor(estimators=[("rf",best_rf),
                                                    ("lgbm",best_lgbm)],
                                        final_estimator=meta,
                                        cv=5,n_jobs=-1)

        # build transformed regressor
        model = TransformedTargetRegressor(regressor=stacking_reg,
                                            transformer=pt)
        
        full_pipe = Pipeline([
            ('target_encoder', ce.TargetEncoder(cols=target_encode)),
            ('preprocessor', preprocessor),
            ('model', model),
        ])
        

        # train the model
        full_pipe.fit(X_train,y_train)
        
        # log model params
        mlflow.log_params(full_pipe.get_params())

        # get the predictions
        y_pred_test = full_pipe.predict(X_test)

        # mean absoulte error
        error = mean_absolute_error(y_test,y_pred_test)

        # log error
        mlflow.log_metric("MAE",error)

        return error

In [21]:
# create optuna study
study = optuna.create_study(direction="minimize")

with mlflow.start_run(run_name="best_model"):
    # optimize the objective function
    study.optimize(objective,n_trials=20,n_jobs=-1,show_progress_bar=True)

    # log the best parameters
    mlflow.log_params(study.best_params)

    # log the best score
    mlflow.log_metric("best_score",study.best_value)

[I 2025-12-12 18:55:37,480] A new study created in memory with name: no-name-9b1537c3-3af9-4b1e-bfdf-1d26f164adcd
  0%|          | 0/20 [00:00<?, ?it/s]c:\Users\soura\OneDrive\Desktop\real-estate-hybrid-app\.venv\Lib\site-packages\sklearn\compose\_column_transformer.py:975: FutureWarning: The parameter `force_int_remainder_cols` is deprecated and will be removed in 1.9. It has no effect. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\soura\OneDrive\Desktop\real-estate-hybrid-app\.venv\Lib\site-packages\sklearn\compose\_column_transformer.py:975: FutureWarning: The parameter `force_int_remainder_cols` is deprecated and will be removed in 1.9. It has no effect. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\soura\OneDrive\Desktop\real-estate-hybrid-app\.venv\Lib\site-packages\sklearn\compose\_column_transformer.py:975: FutureWarning: The parameter `force_int_remainder_cols` is deprecated and will be removed in 1.9. It has no

🏃 View run spiffy-swan-679 at: https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow/#/experiments/4/runs/170dd544d53a47cda60d574b3016c938
🧪 View experiment at: https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow/#/experiments/4


Best trial: 7. Best value: 0.597094:   5%|▌         | 1/20 [08:47<2:46:56, 527.21s/it]

[I 2025-12-12 19:04:26,778] Trial 7 finished with value: 0.597094343335671 and parameters: {'model': 'KNN', 'n_neighbors_knn': 15, 'weights_knn': 'uniform'}. Best is trial 7 with value: 0.597094343335671.


c:\Users\soura\OneDrive\Desktop\real-estate-hybrid-app\.venv\Lib\site-packages\sklearn\compose\_column_transformer.py:975: FutureWarning: The parameter `force_int_remainder_cols` is deprecated and will be removed in 1.9. It has no effect. Leave it to its default value to avoid this warning.
  warnings.warn(


...

[I 2025-12-12 19:20:15,792] Trial 17 finished with value: 0.5781639178254179 and parameters: {'model': 'LR', 'intercept_lr': False}. Best is trial 17 with value: 0.5781639178254179.
🏃 View run flawless-shoat-745 at: https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow/#/experiments/4/runs/da0efc4968d6485e81bd0ab7387a5420
🧪 View experiment at: https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow/#/experiments/4


Best trial: 17. Best value: 0.578164:  95%|█████████▌| 19/20 [24:45<00:40, 40.11s/it]

[I 2025-12-12 19:20:25,558] Trial 18 finished with value: 0.5783522714264484 and parameters: {'model': 'LR', 'intercept_lr': False}. Best is trial 17 with value: 0.5781639178254179.
🏃 View run delightful-ox-114 at: https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow/#/experiments/4/runs/42b9efea61ba498d81ed926589a0b16e
🧪 View experiment at: https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow/#/experiments/4


Best trial: 19. Best value: 0.578153: 100%|██████████| 20/20 [25:00<00:00, 75.04s/it]


[I 2025-12-12 19:20:40,308] Trial 19 finished with value: 0.5781532683324524 and parameters: {'model': 'LR', 'intercept_lr': False}. Best is trial 19 with value: 0.5781532683324524.
🏃 View run best_model at: https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow/#/experiments/4/runs/1f2ab45dd2cc4e63b967bd801a7bcc7b
🧪 View experiment at: https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow/#/experiments/4


In [22]:
study.best_params

{'model': 'LR', 'intercept_lr': False}

In [23]:
study.best_value

0.5781532683324524

In [24]:
 # stacking regressor
stacking_reg = StackingRegressor(estimators=[("rf",best_rf),
                                            ("lgbm",best_lgbm)],
                                final_estimator=LinearRegression(fit_intercept=True),
                                cv=5,n_jobs=-1)

# build transformed regressor
model = TransformedTargetRegressor(regressor=stacking_reg,
                                    transformer=pt)

full_pipe = Pipeline([
    ('target_encoder', ce.TargetEncoder(cols=target_encode)),
    ('preprocessor', preprocessor),
    ('model', model),
])

In [25]:
full_pipe.fit(X_train,y_train)

c:\Users\soura\OneDrive\Desktop\real-estate-hybrid-app\.venv\Lib\site-packages\sklearn\compose\_column_transformer.py:975: FutureWarning: The parameter `force_int_remainder_cols` is deprecated and will be removed in 1.9. It has no effect. Leave it to its default value to avoid this warning.
  warnings.warn(


,steps,"[('target_encoder', ...), ('preprocessor', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,verbose,0
,cols,['locality']
,drop_invariant,False
,return_df,True
,handle_missing,'value'
,handle_unknown,'value'
,min_samples_leaf,20


In [26]:
y_train_pred = full_pipe.predict(X_train)
y_test_pred = full_pipe.predict(X_test)


print("The R2 score of train data is",r2_score(y_train,y_train_pred))
print("The R2 score of test data is",r2_score(y_test,y_test_pred))


print("The MAE of train data is",mean_absolute_error(y_train,y_train_pred))
print("The MAE of test data is",mean_absolute_error(y_test,y_test_pred))

The R2 score of train data is 0.9499306038969537
The R2 score of test data is 0.8830486209603152
The MAE of train data is 0.40998515981298994
The MAE of test data is 0.5789068430576939


In [27]:
round(r2_score(y_test,y_test_pred),2)

0.88

In [28]:
round(r2_score(y_train,y_train_pred),2)

0.95